<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/ipopt_colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:40
🔁 Restarting kernel...


In [1]:
!conda install -c conda-forge cyipopt

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cyipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-mp-3.1.0              |    h2cc385e_1006         1.1 MB  conda-forge
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    certifi-2022.6.15          |   py37h89c1867_0         155 KB  conda-forge
    conda-4.13.0               |   py37h89c1867_1         989 KB  conda-forge
    cyipopt-1.1.0              |   py37hb2fd402_3         135 KB  conda-forge
    future-0.18.2              |   py37h89c1867_5         713 KB  conda-forge


In [2]:
from jax.config import config

# Enable 64 bit floating point precision
config.update("jax_enable_x64", True)

# We use the CPU instead of GPU und mute all warnings if no GPU/TPU is found.
config.update('jax_platform_name', 'cpu')

from cyipopt import minimize_ipopt
from jax import jit, grad, jacrev, jacfwd
import jax.numpy as np



In [3]:

# Test the scipy interface on the Hock & Schittkowski test problem 71:
#
# min x0*x3*(x0+x1+x2)+x2
#
# s.t. x0**2 + x1**2 + x2**2 + x3**2 - 40 = 0
#      x0 * x1 * x2 * x3 - 25 >= 0
#      1 <= x0,x1,x2,x3 <= 5
#
# We evaluate all derivatives (except the Hessian) by algorithmic differentation
# by means of the JAX library.


def objective(x):
    return x[0]*x[3]*np.sum(x[:3]) + x[2]


def eq_constraints(x):
    return np.sum(x**2) - 40


def ineq_constrains(x):
    return np.prod(x) - 25


# jit the functions
obj_jit = jit(objective)
con_eq_jit = jit(eq_constraints)
con_ineq_jit = jit(ineq_constrains)

# build the derivatives and jit them
obj_grad = jit(grad(obj_jit))  # objective gradient
obj_hess = jit(jacrev(jacfwd(obj_jit)))  # objective hessian
con_eq_jac = jit(jacfwd(con_eq_jit))  # jacobian
con_ineq_jac = jit(jacfwd(con_ineq_jit))  # jacobian
con_eq_hess = jacrev(jacfwd(con_eq_jit)) # hessian
con_eq_hessvp = jit(lambda x, v: con_eq_hess(x) * v[0]) # hessian vector-product
con_ineq_hess = jacrev(jacfwd(con_ineq_jit))  # hessian
con_ineq_hessvp = jit(lambda x, v: con_ineq_hess(x) * v[0])  # hessian vector-product

# constraints
# Note that 'hess' is the hessian-vector-product
cons = [
    {'type': 'eq', 'fun': con_eq_jit, 'jac': con_eq_jac, 'hess': con_eq_hessvp},
    {'type': 'ineq', 'fun': con_ineq_jit, 'jac': con_ineq_jac, 'hess': con_ineq_hessvp},
]

# initial guess
x0 = np.array([1.0, 5.0, 5.0, 1.0])

# variable bounds: 1 <= x[i] <= 5
bnds = [(1, 5) for _ in range(x0.size)]

res = minimize_ipopt(obj_jit, jac=obj_grad, hess=obj_hess, x0=x0, bounds=bnds,
                     constraints=cons, options={'disp': 5})

print(res)

     fun: 17.01401727277449
    info: {'x': array([0.99999999, 4.74299964, 3.82114998, 1.37940831]), 'g': array([ 9.94759830e-14, -9.98234027e-09]), 'obj_val': 17.01401727277449, 'mult_g': array([ 0.16146857, -0.55229366]), 'mult_x_L': array([1.08787121e+00, 2.67142698e-12, 3.54589582e-12, 2.63493908e-11]), 'mult_x_U': array([2.49978528e-12, 3.92746433e-11, 8.47959376e-12, 2.76270134e-12]), 'status': 0, 'status_msg': b'Algorithm terminated successfully at a locally optimal point, satisfying the convergence tolerances (can be specified by options).'}
 message: b'Algorithm terminated successfully at a locally optimal point, satisfying the convergence tolerances (can be specified by options).'
    nfev: 9
     nit: 8
    njev: 10
  status: 0
 success: True
       x: array([0.99999999, 4.74299964, 3.82114998, 1.37940831])
